# 0. 介绍
如果有人问学数据分析有没有一条捷径的话? 答案可能就是kaggle.  
        
kaggle上除了很有很多比赛之后,还有很多大大小小的数据集. 这些数据集是极好的练手的素材.   
这些数据集从几K到十几个G的都有, 大家可以按需下载.下载地址 : https://www.kaggle.com/datasets   
推荐大家下载"Only Datasets with Tasks"的数据集, 这些数据集自带任务,各任务下还有大家提交的答案, 这样每个数据集就相当于一个教程,非常有用. 
![图示](img/1.png)
      
       
本次数据集为 Google 应用商店 App 的下载情况,包含七万多条数据.   
下载地址为: https://www.kaggle.com/lava18/google-play-store-apps/tasks?taskId=276


# 1. 数据集预处理

In [1]:
import pandas as pd
import numpy as np
import matplotlib 
import matplotlib.pyplot as plt
import datetime

## 1.1 读取数据

In [2]:
location1 = r'dateset_kaggle/googleplaystore.csv'
df1 = pd.read_csv(location1)
df1.tail()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
10836,Sya9a Maroc - FR,FAMILY,4.5,38,53M,"5,000+",Free,0,Everyone,Education,"July 25, 2017",1.48,4.1 and up
10837,Fr. Mike Schmitz Audio Teachings,FAMILY,5.0,4,3.6M,100+,Free,0,Everyone,Education,"July 6, 2018",1.0,4.1 and up
10838,Parkinson Exercices FR,MEDICAL,NaN,3,9.5M,"1,000+",Free,0,Everyone,Medical,"January 20, 2017",1.0,2.2 and up
10839,The SCP Foundation DB fr nn5n,BOOKS_AND_REFERENCE,4.5,114,Varies with device,"1,000+",Free,0,Mature 17+,Books & Reference,"January 19, 2015",Varies with device,Varies with device
10840,iHoroscope - 2018 Daily Horoscope & Astrology,LIFESTYLE,4.5,398307,19M,"10,000,000+",Free,0,Everyone,Lifestyle,"July 25, 2018",Varies with device,Varies with device


## 1.2 数据格式转换

### 1.2.1 初步查看数据

In [3]:
# 检查数据类型
df1.info()

# 结果看到只有Rating是数值,很多列的数据类型都不对.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10841 entries, 0 to 10840
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   App             10841 non-null  object 
 1   Category        10841 non-null  object 
 2   Rating          9367 non-null   float64
 3   Reviews         10841 non-null  object 
 4   Size            10841 non-null  object 
 5   Installs        10841 non-null  object 
 6   Type            10840 non-null  object 
 7   Price           10841 non-null  object 
 8   Content Rating  10840 non-null  object 
 9   Genres          10841 non-null  object 
 10  Last Updated    10841 non-null  object 
 11  Current Ver     10833 non-null  object 
 12  Android Ver     10838 non-null  object 
dtypes: float64(1), object(12)
memory usage: 1.1+ MB


In [4]:
# 观察一下缺失值的情况
df1.count()  

# 可以看到行数应该是10841.
# Rating列只有9366, 缺少的数据量比较大,后面可能要用平均值填充.
# Type列缺1行数据,可以直接删除.
# CurrentVer列和AndroidVer也缺几条数据,后面具体看看看怎么处理.     

App               10841
Category          10841
Rating             9367
Reviews           10841
Size              10841
Installs          10841
Type              10840
Price             10841
Content Rating    10840
Genres            10841
Last Updated      10841
Current Ver       10833
Android Ver       10838
dtype: int64

### 1.2.2 APP列 : 检查是否有重复值

In [5]:
print( len(df1.App) )
print( len(df1.App.unique()) )

# App列中行数为10841,非重复行有9660,说明重复项很多.
# 贸然删除影响比较大,最后看看怎么处理.

10841
9660


### 1.2.3 处理Category列 : 删除异常值

In [6]:
# df1['Category'].value_counts(dropna=False ) 
# 发现一个异常值Category=1.9, 可以直接删除.

df1 = df1[~df1['Category'].isin(['1.9'])]  # 通过取反来删除

### 1.2.4 处理Rating列 : 用均值填充NaN值

In [7]:
# df1['Rating'].value_counts(dropna=False ) 
# 发现NaN有1474条数据,数据量大,不能直接删除,这里用均值填充NaN.

df1['Rating'].fillna(df1['Rating'].mean(),inplace=True)

### 1.2.5 处理Reviews列 : 改变数据类型

In [8]:
#df1['Reviews'].value_counts(dropna=False ) 

# 转为为float型
df1['Reviews'] = df1['Reviews'].astype('int64')

### 1.2.6 处理Size列 : 替换异常值,统一单位,改为数值型

In [19]:
df1['Size'].value_counts(dropna=False ) 

# 发现这一列的数据问题有:
# 单位有M和K,需要统一单位
# 需要去掉单位,并且转化为数值型
# 共有10840行数据,异常值'Varies with device'有1695个.用均值代替.

Varies with device    1695
11M                    198
12M                    196
14M                    194
13M                    191
                      ... 
523k                     1
624k                     1
253k                     1
444k                     1
245k                     1
Name: Size, Length: 461, dtype: int64

In [22]:
# 1) 统一计量单位, 并去掉单位,同时转化为浮点型

# 选出非Varies with device的数据行,
# 然后将单位统一(k=M/1024), 然后去掉单位,并转化为浮点型.

def convert_K2M(item): 
    if 'k' in item :
        item = round(float(item.replace('k',''))/1024 ,2)
    else:
        item = round(float(item.replace('M','')),2)     
    return item 

df1['Size']= df1[~df1['Size'].isin(['Everyone'])].apply(convert_K2M)

df1['Size'].value_counts(dropna=False ) 

TypeError: cannot convert the series to <class 'float'>

In [ ]:
# 2) 处理异常值

# df1.groupby(['Size']).size()['Varies with device']   
# 返回1695,数据量较大, 最好不要直接删除,此处用均值代替.

df1['Size'] = df1['Size'].replace('Varies with device',df1['Size'].mean()) # 将'Varies with device'转化为'0'
'0'  in  df1['Size'].values.tolist()
df1.groupby(['Size']).size()['0']  

### 1.2.3 

In [23]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10840 entries, 0 to 10840
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   App             10840 non-null  object 
 1   Category        10840 non-null  object 
 2   Rating          10840 non-null  float64
 3   Reviews         10840 non-null  int64  
 4   Size            10840 non-null  object 
 5   Installs        10840 non-null  object 
 6   Type            10839 non-null  object 
 7   Price           10840 non-null  object 
 8   Content Rating  10840 non-null  object 
 9   Genres          10840 non-null  object 
 10  Last Updated    10840 non-null  object 
 11  Current Ver     10832 non-null  object 
 12  Android Ver     10838 non-null  object 
dtypes: float64(1), int64(1), object(11)
memory usage: 1.2+ MB


### 1.2.7 


### 1.2.4 处理Installs列 : 去掉加号和逗号,  改变数据类型

In [ ]:
# 1) 去掉值中的+号和,号
df1['Installs'] = df1['Installs'].replace('+','').str.replace(',','')

In [ ]:
# 2) 转为为float型
df1['Installs'] = df1['Installs'].astype('float')

### 1.2.5 处理Last Updated列 : 改变时间类型

In [ ]:
# 原数据日期格式比较怪,用7-JAN-18来表示2018-1-15. January 7, 2018
# ('January 7, 2018').split(' ') 返回['January', '7,', '2018']

def time_update (item):  
    t1 = item.replace(',','').replace(' ','') #得到类似January72018
    t2 = datetime.datetime.strptime(t1,'%B%d%Y') #%B表示英文月份.
    return t2

df1['Last Updated'] = df1['Last Updated'].apply(time_update )
df1['Last Updated']

### 1.2.1 处理Price列 : 删除异常值,去掉$,转化为数值型

In [ ]:
# 用 df1['Price'].unique() 查看Price列中有哪些不重复值      
# 发现问题有3个:           
# 1)有异常值"Everyone"    
# 2)值有带$单位       
# 3)值都是字符串型,应该是数值型 


# 1) 删除异常值

# 先查看下值为"Everyone"数据量有多少, 评估下删除对整体数据的影响.
df1.groupby(['Price']).size()['Everyone']  #返回1,只有1条'Everyone'数据,所以直接删掉.

#删除'Price'列中"Everyone"所在行, 用取反排除的方法.
df1 = df1[~df1['Price'].isin(['Everyone'])]

df1['Price'].unique()   # 检查是否成功删除

# 2) 将'Price'列中的$去掉
df1['Price'] = df1['Price'].str.strip('$')

# 3) 将'Price'列转化为数字型
df1['Price'] = df1['Price'].astype('float')